# Getting necessary requirements for the experiment

# Requried imports and helper function

In [26]:
IMAGE_SIZE = 256

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install lightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.7/640.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 15.0 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144552 sha256=47821727a12a88f249879e8f2bec95db99f379c2cb865186ebe97956901e67de
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [3]:
!pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade

  Cloning https://github.com/PytorchLightning/lightning-bolts.git (to revision master) to /tmp/pip-req-build-p2u9bzo6
  Running command git clone --filter=blob:none --quiet https://github.com/PytorchLightning/lightning-bolts.git /tmp/pip-req-build-p2u9bzo6
  Resolved https://github.com/PytorchLightning/lightning-bolts.git to commit 4f910f6c9893eaf206eed6b7df545c8a02043a55
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for lightning-bolts: filename=lightning_bolts-0.7.0-py3-none-any.whl size=303900 sha256=44cba743aeafdd7304eb83b7d7f7ed6a73452e61eafbe72c99f4d7418f7d2f09
  Stored in directory: /tmp/pip-ephem-wheel-cache-x80_pb60/wheels/1e/24/be/48c6a7a89feb08b86daf4b48c2b6a8fbb0bedaadbdccd1e758
Successfully built lightning-bolts
  Attempting uninstall: pytorch-lightning
    Found existing installation: pyto

In [4]:
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00


In [2]:
import torch
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import pl_bolts
from pl_bolts.models.autoencoders import VAE
from pl_bolts.models.self_supervised import SimCLR, SwAV
import torch.nn as nn
import torchvision

/usr/local/lib/python3.10/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.10/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveTask] = FeatureMapCon

In [9]:
%cd /content/drive/MyDrive/Proj-DS/

/content/drive/MyDrive/Proj-DS


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
!pwd

/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# Load the Explanation method
from contrastive_corpus_similarity import ContrastiveCorpusSimilarity
from rise import RISE
from utils import get_black_baseline

In [11]:
def imsc(img, *args, quiet=False, lim=None, interpolation='lanczos', **kwargs):
#for visualising images in matplotlib
    if isinstance(img, Image.Image):
        img = pil_to_tensor(img)
    handle = None
    with torch.no_grad():
        if not lim:
            lim = [img.min(), img.max()]
        img = img - lim[0]
        img.mul_(1 / (lim[1] - lim[0]))
        img = torch.clamp(img, min=0, max=1)
        if not quiet:
            bitmap = img.expand(3,
                                *img.shape[1:]).permute(1, 2, 0).cpu().numpy()
    return bitmap

# Get the models

In [28]:
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead
import pytorch_lightning as pl

class SimCLRModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # create a ResNet backbone and remove the classification head
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        hidden_dim = resnet.fc.in_features
        self.projection_head = SimCLRProjectionHead(hidden_dim, hidden_dim, 128)

        self.criterion = NTXentLoss()

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

In [29]:
#load some models

def load_supervised():
    supervised = torchvision.models.resnet50(pretrained=True)
    modules = list(supervised.children())[:-1]
    supervised = nn.Sequential(*modules, nn.Flatten()).to(DEVICE)
    supervised.eval()
    return supervised

def load_swav():
    swav_weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar'
    swav = SwAV.load_from_checkpoint(swav_weight_path, strict=False)
    swav = swav.to(DEVICE)
    swav.eval()
    return swav


def load_simclr_custom():
    simclr = torch.load('./full_model3.pth')
    simclr.eval()
    return simclr

def load_vae():
    vae = VAE(input_height=IMAGE_SIZE).from_pretrained('stl10-resnet18')
    modules = list(vae.children())[:-3]
    vae = nn.Sequential(*modules, nn.Flatten()).to(DEVICE)
    vae.eval()
    return vae

In [30]:
def load_simclr():
    simclr_weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
    simclr = SimCLR.load_from_checkpoint(simclr_weight_path, strict=False)
    simclr = simclr.to(DEVICE)
    simclr.eval()
    return simclr


In [31]:
#dowload models

load_supervised()
load_swav()
load_simclr()
load_vae()
load_simclr_custom()

print('done')

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
Downloading: "https://pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/checkpoints/epoch%3D89.ckpt" to /root/.cache/torch/hub/checkpoints/epoch%3D89.ckpt
100%|██████████| 230M/230M [00:03<00:00, 63.4MB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded chec

done


In [32]:
def load_img_from_drive(img_name, shape=IMAGE_SIZE):
  #loads two4two images from personal drive (after they have been uploaded and unzipped)
    img = Image.open('../242/test/{}'.format(img_name)).convert('RGB')

    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((shape, shape)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225]),
    ])

    x = transform(img).unsqueeze(0)

    return x.to(DEVICE)

In [33]:
def load_mask_from_drive(mask_name, shape=IMAGE_SIZE):
  #loads two4two masks from personal drive (after they have been uploaded and unzipped)
    mask = cv2.imread('../242/test/{}'.format(mask_name))

    target_transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), torchvision.transforms.Resize((shape, shape))])
    mask = target_transform(mask).sum(dim=0).unsqueeze(dim=0)

    mask = torch.where(mask > 0, torch.tensor(1), torch.tensor(0))

    return mask

In [34]:
def to_np(x):
    return x.cpu().detach().numpy()

# Classes to perform COCOA

In [42]:
from functools import partial

get_baseline = partial(
    get_black_baseline
)



# Get data for the experiment

In [43]:
data_path = "./mixed/"

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the images to a specific size
    transforms.ToTensor(),  # Convert the images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),

])

# Load the images from the directory using the ImageFolder dataset
dataset = ImageFolder(root=data_path, transform=transform)

# Create a data loader to iterate over the dataset in batches
batch_size = 32
data_loader_ref = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [44]:
data_path = "./noise/"

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the images to a specific size
    transforms.ToTensor(),  # Convert the images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
])

# Load the images from the directory using the ImageFolder dataset
dataset = ImageFolder(root=data_path, transform=transform)

# Create a data loader to iterate over the dataset in batches
batch_size = 32
data_loader_foil = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [59]:
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision.transforms.functional import pil_to_tensor
from torchvision.transforms import ToTensor, Normalize, Resize

import tqdm.notebook as tqdm

import matplotlib.gridspec as gridspec

import numpy as np

from PIL import Image
import cv2

from matplotlib import pyplot as plt

from pl_bolts.models.self_supervised import SimCLR, SwAV
from pl_bolts.models.autoencoders import VAE

import warnings
warnings.filterwarnings("ignore")
from captum.attr import IntegratedGradients, GradientShap


In [62]:
model_super = load_supervised()
model_sim = load_simclr()
model_swav = load_swav()
model_vae = load_vae()
model_sim_custom = load_simclr_custom()

ccs_super = ContrastiveCorpusSimilarity(
        encoder=model_super.to(DEVICE),
        corpus_dataloader=data_loader_ref,
        foil_dataloader=data_loader_foil,
        normalize=True,
        batch_size=32
      )

ccs_sim = ContrastiveCorpusSimilarity(
        encoder=model_sim.to(DEVICE),
        corpus_dataloader=data_loader_ref,
        foil_dataloader=data_loader_foil,
        normalize=True,
        batch_size=32,
      )

ccs_swav = ContrastiveCorpusSimilarity(
        encoder=model_swav.to(DEVICE),
        corpus_dataloader=data_loader_ref,
        foil_dataloader=data_loader_foil,
        normalize=True,
        batch_size=32
      )

ccs_vae = ContrastiveCorpusSimilarity(
        encoder=model_vae.to(DEVICE),
        corpus_dataloader=data_loader_ref,
        foil_dataloader=data_loader_foil,
        normalize=True,
        batch_size=32
      )

ccs_sim_custom = ContrastiveCorpusSimilarity(
        encoder=model_sim_custom.to(DEVICE),
        corpus_dataloader=data_loader_ref,
        foil_dataloader=data_loader_foil,
        normalize=True,
        batch_size=32
      )

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

In [63]:
ccs_list = [ccs_super, ccs_sim, ccs_swav, ccs_vae, ccs_sim_custom]

# RISE Experiment

In [64]:
#on Google Colab this takes a looooong time (Google kicks you out way before it finishes)
#for our 1st preliminary results we used the rolling average and let it print after every image processed
#need to do this on the TU Berlin compute cluster completly

import os

list_images = sorted(os.listdir('../242/test'))

model_name_list = ['Supervised', 'SimCLR', 'SwAV', 'VAE']#, 'Custom SimCLR']
model_list = [load_supervised, load_simclr, load_swav, load_vae]#, load_simclr_custom]
mask_bs = 100
num_batches = 30

tk_list = []
pg_list = []
rra_list = []

print(model_name_list)
#we have a total of 5k images and 5k masks for the images
#iterate over images + masks
for i in range(0,4650):
    attr_list = []
    tk_indv_list = []
    pg_indv_list = []
    rra_indiv_list = []
    image_name = list_images[i*2]
    mask_name = list_images[(i*2)+1]
    x = load_img_from_drive(image_name)
    baseline = get_baseline(x)
    mask = load_mask_from_drive(mask_name)

    #iterate over models
    for model_loader, model_name, ccs in zip(model_list, model_name_list, ccs_list):

#         model = model_loader()

        with torch.no_grad():

            rise = RISE(ccs)
            attr = rise.attribute(
                    x.cuda(),
                    (8,8),
                    baselines=baseline.cuda(),
                    mask_prob=0.5,
                    n_samples=3000,
                    normalize_by_mask_prob=True,
                )

        attr_list.append(attr)


  #iterate over models' results
    for j in range(len(model_list)):
        s=to_np(mask.squeeze())
        a=to_np(attr_list[j].mean(dim=1).cpu().detach().squeeze())

        k=1000
        #prepare shapes
        s = s.astype(bool)
        top_k_binary_mask = np.zeros(a.shape)

        #top-k intersection
        #sort and create masks
        sorted_indices = np.argsort(a, axis=None)
        np.put_along_axis(top_k_binary_mask, sorted_indices[-k:], 1, axis=None)
        top_k_binary_mask = top_k_binary_mask.astype(bool)
        tki = 1.0 / k * np.sum(np.logical_and(s, top_k_binary_mask))

        #pointing game
        #find indices with max value
        a_pg = a.copy().flatten()
        s_pg = to_np(mask.squeeze()).flatten().astype(bool)
        max_index = np.argwhere(a_pg == np.max(a_pg))
        #check if maximum of explanation is on target object class.
        hit = np.any(s_pg[max_index])

        #relevance rank accuracy
        s = to_np(mask.squeeze())
        s = np.where(s.flatten().astype(bool))[0]
        #size of the ground truth mask
        n = len(s)
        #sort in descending order
        a_sorted = np.argsort(a.flatten())[-int(n) :]
        #calculate hits
        hits = len(np.intersect1d(s, a_sorted))
        if hits != 0:
            rank_accuracy = hits / float(n)
        else:
            rank_accuracy = 0.0

        tk_indv_list.append(tki)
        pg_indv_list.append(np.float(hit))
        rra_indiv_list.append(rank_accuracy)
    tk_list.append(tk_indv_list)
    pg_list.append(pg_indv_list)
    rra_list.append(rra_indiv_list)
    if i % 1 == 0: #print rolling mean after every **1** images processed.
        print(i)
        print('Pointing Game:')
        print(np.array(pg_list).mean(axis=0))
        print('Top-K:')
        print(np.array(tk_list).mean(axis=0))
        print('Relevance Rank Acc.:')
        print(np.array(rra_list).mean(axis=0))

['Supervised', 'SimCLR', 'SwAV', 'VAE']
0
Pointing Game:
[1. 1. 1. 0.]
Top-K:
[0.579 0.877 0.941 0.   ]
Relevance Rank Acc.:
[0.41124031 0.44467054 0.62141473 0.0126938 ]


KeyboardInterrupt: ignored

# Integrated Gradients Experiment

In [66]:
#on Google Colab this takes a looooong time (Google kicks you out way before it finishes)
#for our 1st preliminary results we used the rolling average and let it print after every image processed
#need to do this on the TU Berlin compute cluster completly

# from google.colab import drive
import os

# drive.mount('/content/drive')
list_images = sorted(os.listdir('../242/test'))

model_name_list = ['Supervised', 'SimCLR', 'SwAV', 'VAE', 'Custom SimCLR']
model_list = [load_supervised, load_simclr, load_swav, load_vae, load_simclr_custom]
mask_bs = 100
num_batches = 30

tk_list = []
pg_list = []
rra_list = []

print(model_name_list)
#we have a total of 5k images and 5k masks for the images
#iterate over images + masks
for i in range(0,4650):
    attr_list = []
    tk_indv_list = []
    pg_indv_list = []
    rra_indiv_list = []
    image_name = list_images[i*2]
    mask_name = list_images[(i*2)+1]
    x = load_img_from_drive(image_name)
    baseline = get_baseline(x)
    mask = load_mask_from_drive(mask_name)

    #iterate over models
    for model_loader, model_name, ccs in zip(model_list, model_name_list, ccs_list):

        model = model_loader()

        with torch.no_grad():

            ig = IntegratedGradients(ccs)
            attr = ig.attribute(
                    x.cuda(),
                    baselines=baseline.cuda(),
                )

        attr_list.append(attr)


  #iterate over models' results
    for j in range(len(model_list)):
        s=to_np(mask.squeeze())
        a=to_np(attr_list[j].mean(dim=1).cpu().detach().squeeze())

        k=1000
        #prepare shapes
        s = s.astype(bool)
        top_k_binary_mask = np.zeros(a.shape)

        #top-k intersection
        #sort and create masks
        sorted_indices = np.argsort(a, axis=None)
        np.put_along_axis(top_k_binary_mask, sorted_indices[-k:], 1, axis=None)
        top_k_binary_mask = top_k_binary_mask.astype(bool)
        tki = 1.0 / k * np.sum(np.logical_and(s, top_k_binary_mask))

        #pointing game
        #find indices with max value
        a_pg = a.copy().flatten()
        s_pg = to_np(mask.squeeze()).flatten().astype(bool)
        max_index = np.argwhere(a_pg == np.max(a_pg))
        #check if maximum of explanation is on target object class.
        hit = np.any(s_pg[max_index])

        #relevance rank accuracy
        s = to_np(mask.squeeze())
        s = np.where(s.flatten().astype(bool))[0]
        #size of the ground truth mask
        n = len(s)
        #sort in descending order
        a_sorted = np.argsort(a.flatten())[-int(n) :]
        #calculate hits
        hits = len(np.intersect1d(s, a_sorted))
        if hits != 0:
            rank_accuracy = hits / float(n)
        else:
            rank_accuracy = 0.0

        tk_indv_list.append(tki)
        pg_indv_list.append(np.float(hit))
        rra_indiv_list.append(rank_accuracy)
    tk_list.append(tk_indv_list)
    pg_list.append(pg_indv_list)
    rra_list.append(rra_indiv_list)
    if i % 1 == 0: #print rolling mean after every **1** images processed.
        print(i)
        print('Pointing Game:')
        print(np.array(pg_list).mean(axis=0))
        print('Top-K:')
        print(np.array(tk_list).mean(axis=0))
        print('Relevance Rank Acc.:')
        print(np.array(rra_list).mean(axis=0))

['Supervised', 'SimCLR', 'SwAV', 'VAE', 'Custom SimCLR']


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

0
Pointing Game:
[0. 0. 1. 0. 0.]
Top-K:
[0.306 0.244 0.276 0.08  0.118]
Relevance Rank Acc.:
[0.22083333 0.17674419 0.16337209 0.10406977 0.16104651]


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

1
Pointing Game:
[0.5 0.  1.  0.5 0.5]
Top-K:
[0.342  0.233  0.4515 0.175  0.4095]
Relevance Rank Acc.:
[0.23429054 0.18214769 0.22588138 0.17723963 0.25901425]


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

2
Pointing Game:
[0.33333333 0.         1.         0.33333333 0.33333333]
Top-K:
[0.382      0.219      0.44366667 0.18833333 0.284     ]
Relevance Rank Acc.:
[0.248211   0.18369421 0.23169258 0.17334392 0.18735699]


KeyboardInterrupt: ignored

# GradientShap Experiment

In [67]:
#on Google Colab this takes a looooong time (Google kicks you out way before it finishes)
#for our 1st preliminary results we used the rolling average and let it print after every image processed
#need to do this on the TU Berlin compute cluster completly

# from google.colab import drive
import os

# drive.mount('/content/drive')
list_images = sorted(os.listdir('../242/test'))

model_name_list = ['Supervised', 'SimCLR', 'SwAV', 'VAE']#, 'Custom SimCLR']
model_list = [load_supervised, load_simclr, load_swav, load_vae]#, load_simclr_custom]
mask_bs = 100
num_batches = 30

tk_list = []
pg_list = []
rra_list = []

print(model_name_list)
#we have a total of 5k images and 5k masks for the images
#iterate over images + masks
for i in range(0,4650):
    attr_list = []
    tk_indv_list = []
    pg_indv_list = []
    rra_indiv_list = []
    image_name = list_images[i*2]
    mask_name = list_images[(i*2)+1]
    x = load_img_from_drive(image_name)
    baseline = get_baseline(x)
    mask = load_mask_from_drive(mask_name)

    #iterate over models
    for model_loader, model_name, ccs in zip(model_list, model_name_list, ccs_list):

        model = model_loader()

        with torch.no_grad():

            gs = GradientShap(ccs)
            attr = gs.attribute(
                    x.cuda(),
                    baselines=baseline.cuda(),
                    n_samples=50,
                    stdevs=0.2
                )

        attr_list.append(attr)


  #iterate over models' results
    for j in range(len(model_list)):
        s=to_np(mask.squeeze())
        a=to_np(attr_list[j].mean(dim=1).cpu().detach().squeeze())

        k=1000
        #prepare shapes
        s = s.astype(bool)
        top_k_binary_mask = np.zeros(a.shape)

        #top-k intersection
        #sort and create masks
        sorted_indices = np.argsort(a, axis=None)
        np.put_along_axis(top_k_binary_mask, sorted_indices[-k:], 1, axis=None)
        top_k_binary_mask = top_k_binary_mask.astype(bool)
        tki = 1.0 / k * np.sum(np.logical_and(s, top_k_binary_mask))

        #pointing game
        #find indices with max value
        a_pg = a.copy().flatten()
        s_pg = to_np(mask.squeeze()).flatten().astype(bool)
        max_index = np.argwhere(a_pg == np.max(a_pg))
        #check if maximum of explanation is on target object class.
        hit = np.any(s_pg[max_index])

        #relevance rank accuracy
        s = to_np(mask.squeeze())
        s = np.where(s.flatten().astype(bool))[0]
        #size of the ground truth mask
        n = len(s)
        #sort in descending order
        a_sorted = np.argsort(a.flatten())[-int(n) :]
        #calculate hits
        hits = len(np.intersect1d(s, a_sorted))
        if hits != 0:
            rank_accuracy = hits / float(n)
        else:
            rank_accuracy = 0.0

        tk_indv_list.append(tki)
        pg_indv_list.append(np.float(hit))
        rra_indiv_list.append(rank_accuracy)
    tk_list.append(tk_indv_list)
    pg_list.append(pg_indv_list)
    rra_list.append(rra_indiv_list)
    if i % 1 == 0: #print rolling mean after every **1** images processed.
        print(i)
        print('Pointing Game:')
        print(np.array(pg_list).mean(axis=0))
        print('Top-K:')
        print(np.array(tk_list).mean(axis=0))
        print('Relevance Rank Acc.:')
        print(np.array(rra_list).mean(axis=0))

['Supervised', 'SimCLR', 'SwAV', 'VAE']


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

0
Pointing Game:
[0. 1. 1. 0.]
Top-K:
[0.655 0.437 0.653 0.085]
Relevance Rank Acc.:
[0.27800388 0.22897287 0.25494186 0.10746124]


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

1
Pointing Game:
[0.5 1.  1.  0.5]
Top-K:
[0.6995 0.523  0.6925 0.152 ]
Relevance Rank Acc.:
[0.29942372 0.24926326 0.27350901 0.15851154]


...

33
Pointing Game:
[0.91176471 0.88235294 0.94117647 0.5       ]
Top-K:
[0.79529412 0.59555882 0.72867647 0.21091176]
Relevance Rank Acc.:
[0.33770624 0.28056091 0.30859742 0.19147404]


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/vae/vae-stl10/ch

34
Pointing Game:
[0.91428571 0.88571429 0.94285714 0.48571429]
Top-K:
[0.79357143 0.59497143 0.72751429 0.21108571]
Relevance Rank Acc.:
[0.33725835 0.27962092 0.30800232 0.19273862]


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.4 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v0.9.1rc3 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar`


KeyboardInterrupt: ignored